# Day 12

## Puzzle 1

In [36]:
with open('../inputs/adventofcode.com_2022_day_12_input.txt', 'r') as f:
    data = f.read().splitlines()

print(f'The file contains {len(data)} lines.')

The file contains 41 lines.


In [37]:
# Translate lowercase to number
import string
num = 0
char2num = {}
for char in string.ascii_lowercase:
    num += 1
    char2num[char] = num
print(char2num)

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [38]:
import numpy as np

# Create the board with the corresponding height for the letters
start = [-1, -1]
end = [-1, -1]
board = np.zeros((int(len(data)), int(len(data[0]))))

for i, line in enumerate(data):
    for j, char in enumerate(line):
        if char == 'S':
            start = [i, j]
            board[i, j] = 0
        elif char == 'E':
            end = [i, j]
            board[i, j] = 26
        else:
            board[i, j] = char2num[char]

print(f'{start=}, {end=}')
print(board)
print(board.shape)

start=[20, 0], end=[20, 120]
[[1. 2. 3. ... 1. 1. 1.]
 [1. 2. 3. ... 1. 1. 1.]
 [1. 2. 3. ... 1. 1. 1.]
 ...
 [1. 2. 3. ... 3. 1. 1.]
 [1. 2. 3. ... 1. 1. 1.]
 [1. 2. 3. ... 1. 1. 1.]]
(41, 143)


In [39]:
# Helpper function to draw the path
# 1 denores the path, 0 the rest
def draw_path(board, current_node, file_name='path.txt'):
    path = np.zeros(board.shape)
    while current_node.parent:
        path[current_node.row, current_node.col] = 1
        current_node = current_node.parent

    # Save to file to visualize
    with open(file_name, 'w') as f:
        for i in range(path.shape[0]):
            for j in range(path.shape[1]):
                f.write(str(int(path[i, j])))
            f.write('\n')

To solve this puzzle I implemented the A* algorithm to go from the starting position to the goal. The explanation for this algorithm can be found [here](https://en.wikipedia.org/wiki/A*_search_algorithm).

Note: The heuristics used is the Manhattan distance.

In [40]:
# Class to represent the nodes (cells) in the board
class Node():
    def __init__(self, coords, g, end, parent):
        self.row = coords[0]
        self.col = coords[1]
        self.g = g
        self.h = abs(end[0] - self.row) + abs(end[1] - self.col)
        self.parent = parent

    def f(self):
        return self.g + self.h

    def __repr__(self):
        return f'({self.row}, {self.col}, {self.f()})'
    
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def update_parent(self, new_parent):
        self.parent = new_parent
        

In [41]:
# Start and end nodes
start_node = Node(start, 0, end, None)
end_node = Node(end, 0, end, None)

# Initialize the open set with the start node, and the close list empty
open_set = [start_node]
closed_list = []

while True:
    # find the node with the lowest f score
    f_scores = [node.f() for node in open_set]
    idx = np.argmin(f_scores)
    current_node = open_set.pop(idx)
    row, col, g = current_node.row, current_node.col, current_node.g

    # if we have reached the end, we are done
    if current_node == end_node:
        print('We are done!')
        print(f'The goal was reached in {current_node.g} steps.')
        break

    else:
        # put the current node in the closed list and look at all of its neighbors
        closed_list.append(current_node)

        # look at all of its neighbors (right, down, left, up)
        for n,m in [[0, 1], [1, 0], [0, -1], [-1, 0]]:
            # A neighbor is valid if it is not outside the board and the difference in height is at most 1
            if row+n >= 0 and row+n < board.shape[0] and col+m >= 0 and col+m < board.shape[1] and board[row+n, col+m] - board[row, col] <= 1: 
                # if the neighbor has lower g value than current and is in the closed list
                neighbor = Node([row+n, col+m], g+1, end, current_node)
                if neighbor in closed_list:
                    idx = closed_list.index(neighbor)
                    if closed_list[idx].g > neighbor.g:
                        closed_list[idx].g = neighbor.g
                        closed_list[idx].update_parent(current_node)

                # else if current g value is lower and this neighbor is in the open list
                elif neighbor in open_set:
                    idx = open_set.index(neighbor)
                    if open_set[idx].g > neighbor.g:
                        open_set[idx].g = neighbor.g
                        open_set[idx].update_parent(current_node)

                # else if this neighbor is not in either list
                else:
                    open_set.append(neighbor)
                    neighbor.g = g + 1


We are done!
The goal was reached in 468 steps.


In [42]:
draw_path(board, current_node, 'puzzle1_path.txt')

## Part 2

To solve this second part, instead of going up from all the possible cells with height ```a```, we start at the top and go down. 

In this case, since we will invert the direction, it is important to notice that:
* We can only go down one at a time
* We can go up more than one at a time

In [43]:
# Since we don't know the ending position, we don't use heuristics
class Node():
    def __init__(self, coords, g, parent):
        self.row = coords[0]
        self.col = coords[1]
        self.g = g
        self.parent = parent

    def __repr__(self):
        return f'({self.row}, {self.col}, {self.f()})'
    
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def update_parent(self, new_parent):
        self.parent = new_parent

In [44]:
start_node = Node(end, 0, None)

open_set = [start_node]
closed_list = []

while True:
    # find the node with the lowest g score
    g_scores = [node.g for node in open_set]
    idx = np.argmin(g_scores)
    current_node = open_set.pop(idx)
    row, col, g = current_node.row, current_node.col, current_node.g

    # if we have reached the end, we are done
    if board[row, col] == 1:
        print('We are done!')
        print(f'The goal was reached in {current_node.g} steps.')
        break

    else:
        # put the current node in the closed list and look at all of its neighbors
        closed_list.append(current_node)
        # look at all of its neighbors (right, down, left, up)
        for n,m in [[0, 1], [1, 0], [0, -1], [-1, 0]]:
            # A neighbor is valid if it is not outside the board and the difference in height is at most 1 (go down one at a time)
            if row+n >= 0 and row+n < board.shape[0] and col+m >= 0 and col+m < board.shape[1] and board[row+n, col+m] - board[row, col] >= -1: 
                # if the neighbor has lower g value than current and is in the closed list
                
                neighbor = Node([row+n, col+m], g+1, current_node)
                if neighbor in closed_list:
                    idx = closed_list.index(neighbor)
                    if closed_list[idx].g > neighbor.g:
                        closed_list[idx].g = neighbor.g
                        closed_list[idx].update_parent(current_node)

                # else if current g value is lower and this neighbor is in the open list
                elif neighbor in open_set:
                    idx = open_set.index(neighbor)
                    if open_set[idx].g > neighbor.g:
                        open_set[idx].g = neighbor.g
                        open_set[idx].update_parent(current_node)
                        
                # else if this neighbor is not in both lists
                else:
                    # print(f'Node {neighbor} is not in either list. Adding to open set.')
                    open_set.append(neighbor)
                    neighbor.g = g + 1

We are done!
The goal was reached in 459 steps.


In [45]:
draw_path(board, current_node, 'puzzle2_path.txt')